# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [72]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [73]:
cities = "../output_data/cities.csv"

# read file into data frame and print first five rows
city_weather = pd.read_csv(cities)
city_weather.head()

,Unnamed: 0,City,Latitude,Longitude,Country,Temperature,% Humidity,% Cloudiness,Wind Speed (mph)
0,0,mataura,-46.1927,168.8643,NZ,47.48,97,9,2.35
1,1,brae,60.3964,-1.3530,GB,43.12,78,100,10.36
2,2,maldonado,-34.6667,-54.9167,UY,61.99,55,0,13.47
3,3,rikitea,-23.1203,-134.9692,PF,77.97,69,31,18.25
4,4,bluff,-46.6000,168.3333,NZ,47.80,86,21,11.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [74]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [85]:
# bring in location data pairs, humidity data, define map and layer, and add layer to map

# store latitude and longitude in locations
locations = city_weather[["Latitude", "Longitude"]]
humidity= city_weather["% Humidity"]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=2)
# add layer
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [78]:
# reduce data frame to cities with temps in between 70 and 85 degrees F, with cloudiness less than 10% and Humidity less than 50%
vacation_weather = city_weather[(city_weather["Temperature"] < 85) & (city_weather["Temperature"] > 70) & (city_weather["% Cloudiness"] <10) & (city_weather["% Humidity"] <50)]
# drop null values
vacation_weather.dropna(inplace = True) 
vacation_weather

/Users/taylorbohl/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Unnamed: 0,City,Latitude,Longitude,Country,Temperature,% Humidity,% Cloudiness,Wind Speed (mph)
17,17,adrar,20.5022,-10.0711,MR,79.75,16,0,10.78
151,151,aswan,24.0934,32.9070,EG,81.70,7,0,6.91
347,347,ojhar,20.1000,73.9333,IN,78.51,33,2,8.57
361,361,sakakah,29.9697,40.2064,SA,72.97,20,0,16.62
485,485,midland,31.9974,-102.0779,US,82.81,12,0,13.80
517,517,rafaela,-31.2503,-61.4867,AR,73.98,19,0,5.99


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
# create dataframe and empty column
hotel_df = vacation_weather[["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""

# print data frame with empty column
hotel_df

/var/folders/h1/xb5j3d_136j05hmpd9xpx6580000gn/T/ipykernel_46092/2951094569.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitude,Hotel Name
17,adrar,MR,20.5022,-10.0711,
151,aswan,EG,24.0934,32.9070,
347,ojhar,IN,20.1000,73.9333,
361,sakakah,SA,29.9697,40.2064,
485,midland,US,31.9974,-102.0779,
517,rafaela,AR,-31.2503,-61.4867,


In [80]:
# loop through dataframe and add nearby hotel result with google API nearby search

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    #access each column for lat,lng pairs in parameters
    lat = row["Latitude"]
    lng = row["Longitude"]
    params = {
   "location": f'{lat},{lng}',
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

    # assemble url and make API request
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response["results"]
    
    # add hotel name to data frame
    try: 
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "n/a"
        
# print data frame with Hotel Name column filled       
hotel_df

/Users/taylorbohl/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Latitude,Longitude,Hotel Name
17,adrar,MR,20.5022,-10.0711,n/a
151,aswan,EG,24.0934,32.9070,Nuba Nile
347,ojhar,IN,20.1000,73.9333,Aurangabad Deogaon Rangari
361,sakakah,SA,29.9697,40.2064,Fakhamat Al Orjoana Apartment
485,midland,US,31.9974,-102.0779,DoubleTree by Hilton Hotel Midland Plaza
517,rafaela,AR,-31.2503,-61.4867,Parra Hotel & Suites


In [81]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [82]:
# add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout)

# assign the marker layer to locations
markers = gmaps.marker_layer(locations)

# add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…